In [33]:
_ = ! pip install web3

In [81]:
import os, sys

from datetime import datetime, timedelta
import time

import requests, json
from web3 import Web3, HTTPProvider

from eth_utils import to_bytes
from eth_keys import keys
from eth_account import Account
from eth_account.messages import SignableMessage

In [82]:
w3 = Web3(HTTPProvider("https://rpc.ankr.com/eth/1e5f9fd3e49ecec581e00815ea1eb41bc459248fd3dcf67716ff3237cc3ab68b"))
# https://1rpc.io/eth

In [83]:
source_token = "0x1f9840a85d5af5bf1d1762f925bdaddc4201f984" # UniSwap
destination_token = "0xdac17f958d2ee523a2206206994597c13d831ec7" # USDT

source_amount = 1000000000000000000
slippage = int(0.05 * 10000) # Convert to bps

In [86]:
eth_wallet = Account.from_key("e5ecbcfc71ad24a2131953ef9a66e4b7ed8ca9a10ffd92104e106fab1f0af4f7")

In [87]:
# 0.  One fresh quote – DO NOT fetch another one before sending
quote  = requests.get(
    "https://api.0x.org/swap/permit2/quote",
    params={
        "chainId": 1,
        "sellToken": source_token,
        "buyToken":  destination_token,
        "sellAmount": source_amount,
        "taker": eth_wallet.address,
        "slippageBps": slippage,
    },
    headers={
        "0x-version" : "v2",
        "0x-api-key" : "50cbb531-4fda-4464-acb4-5e6b5bf0e2d6"
    },
    timeout=10,
).json()

In [88]:
ERC20_ABI: list[dict] = [
    {
        "constant": True,
        "inputs": [{"name": "owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "type": "function",
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "type": "function",
    },
]

erc20 = w3.eth.contract(address=Web3.to_checksum_address(source_token), abi=ERC20_ABI)

sell_amount   = int(quote["sellAmount"])
permit_amount = int(quote["permit2"]["eip712"]["message"]["permitted"]["amount"])
owner_balance = int(erc20.functions.balanceOf(eth_wallet.address).call())

print ("sellAmount        :", sell_amount)
print ("permit.amount     :", permit_amount)
print ("wallet UNI balance:", owner_balance)
print ()
print ("gas fee:", (1.0e-18) * int(quote['transaction']['gas']) * int(quote['transaction']['gasPrice']), "ETH")

sellAmount        : 1000000000000000000
permit.amount     : 1000000000000000000
wallet UNI balance: 57116026654249141437

gas fee: 0.00047935132397940904 ETH


In [ ]:
digest = bytes.fromhex(quote["permit2"]["hash"][2:])
priv   = keys.PrivateKey(eth_wallet.key)
sig    = priv.sign_msg_hash(digest)

sig_bytes = sig.to_bytes()
if sig_bytes[-1] in (0, 1):  # old eth-keys
    sig_bytes = sig_bytes[:-1] + bytes([sig_bytes[-1] + 27])

data_hex = quote["transaction"]["data"]
data     = bytes.fromhex(data_hex[2:])

zero97 = b"\x00" * (32 + 65)
i = data.rfind(zero97)
if i == -1:
    raise ValueError("Permit2 placeholder not found – API layout changed?")

patched = (
    data[:i]
    + len(sig_bytes).to_bytes(32, "big")  # 65
    + sig_bytes
    + data[i + len(zero97):]
)

calldata = "0x" + patched.hex()

# Sanity checks
assert len(sig_bytes) == 65
assert int.from_bytes(patched[i:i+32], "big") == 65, "length word != 65"
assert patched.rfind(zero97) < i, "another zero placeholder remains after injected signature"

In [93]:
# 4. Build, sign, send
tx: dict = {
    "chainId":  1,
    "to":       Web3.to_checksum_address(quote["transaction"]["to"]),
    "data":     Web3.to_hex(calldata),
    "value":    int(quote["transaction"]["value"]),
    "gas":      int(quote["transaction"]["gas"]),
    "nonce":    w3.eth.get_transaction_count(eth_wallet.address),
}

# Prefer EIP-1559 if present
if ("maxFeePerGas" in quote["transaction"]) and ("maxPriorityFeePerGas" in quote["transaction"]):
    tx["type"] = 2
    tx["maxFeePerGas"] = int(quote["transaction"]["maxFeePerGas"])
    tx["maxPriorityFeePerGas"] = int(quote["transaction"]["maxPriorityFeePerGas"])
else:
    tx["gasPrice"] = int(quote["transaction"]["gasPrice"])

# Pass through optional fields
if "accessList" in quote["transaction"]:
    tx["accessList"] = quote["transaction"]["accessList"]

# Only for eth_call / estimateGas preflight
tx = dict(tx, **{"from": eth_wallet.address})

tx_signed = eth_wallet.sign_transaction(tx)
tx_hash = w3.eth.send_raw_transaction(tx_signed.raw_transaction)

In [94]:
# 5. Check result
tx_hex = w3.to_hex(tx_hash)

print("↗ broadcast :", tx_hex)
print("⏳ waiting …")
print("✅ status   :", w3.eth.wait_for_transaction_receipt(tx_hash).status)  # 1 = success

↗ broadcast : 0x6162d10acd4d7be28a5680f570c1acee7023d780c15f3839124b5b0ef9f6b75c
⏳ waiting …
✅ status   : 0


In [95]:
print ("https://eth.blockscout.com/tx/" + tx_hex)

https://eth.blockscout.com/tx/0x6162d10acd4d7be28a5680f570c1acee7023d780c15f3839124b5b0ef9f6b75c


In [96]:
quote

{'blockNumber': '22991588',
 'buyAmount': '10431648',
 'buyToken': '0xdac17f958d2ee523a2206206994597c13d831ec7',
 'fees': {'integratorFee': None,
  'zeroExFee': {'amount': '15674',
   'token': '0xdac17f958d2ee523a2206206994597c13d831ec7',
   'type': 'volume'},
  'gasFee': None},
 'issues': {'allowance': {'actual': '0',
   'spender': '0x000000000022d473030f116ddee9f6b43ac78ba3'},
  'balance': None,
  'simulationIncomplete': False,
  'invalidSourcesPassed': []},
 'liquidityAvailable': True,
 'minBuyAmount': '9910692',
 'permit2': {'type': 'Permit2',
  'hash': '0x6fe93002f53146ea1679781b442ee8b551afad6e8a626d953a86f33b2be5d2f2',
  'eip712': {'types': {'PermitTransferFrom': [{'name': 'permitted',
      'type': 'TokenPermissions'},
     {'name': 'spender', 'type': 'address'},
     {'name': 'nonce', 'type': 'uint256'},
     {'name': 'deadline', 'type': 'uint256'}],
    'TokenPermissions': [{'name': 'token', 'type': 'address'},
     {'name': 'amount', 'type': 'uint256'}],
    'EIP712Domain': 